In [3]:
from google.colab import files
uploaded = files.upload()  # Upload your Assignment15.db file


Saving Assignment 15.sql to Assignment 15.sql


In [12]:
!pip install mysql-connector-python

!pip install pandas scikit-surprise
!pip uninstall -y scikit-surprise numpy
!pip install numpy==1.23.5
!pip install scikit-surprise



Found existing installation: scikit-surprise 1.1.4
Uninstalling scikit-surprise-1.1.4:
  Successfully uninstalled scikit-surprise-1.1.4
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 88.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy

  Using cached scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl


In [7]:
# 1. Imports
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# 2. Load CSVs (already uploaded in file panel)
ratings_df = pd.read_csv("/content/ratings.csv")
movies_df = pd.read_csv("/content/movies.csv")
users_df = pd.read_csv("/content/users.csv")
genres_df = pd.read_csv("/content/genres.csv")

# 3. Train model using Surprise SVD
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings_df[['user_id', 'movie_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

model = SVD()
model.fit(trainset)

# 4. Recommend top 5 movies for a specific user
def get_top_5_for_user(user_id):
    all_movie_ids = movies_df['movie_id'].tolist()
    rated = ratings_df[ratings_df.user_id == user_id]['movie_id'].tolist()
    unrated = [mid for mid in all_movie_ids if mid not in rated]

    preds = [model.predict(user_id, mid) for mid in unrated]
    top_5 = sorted(preds, key=lambda x: x.est, reverse=True)[:5]

    return [(movies_df[movies_df.movie_id == p.iid]['title'].values[0], round(p.est, 2)) for p in top_5]

# 5. Print recommendations for ALL users
print("🎯 Personalized Top 5 Recommendations for Each User:\n")
for uid in users_df['user_id']:
    recs = get_top_5_for_user(uid)
    print(f"User {uid}:")
    for title, score in recs:
        print(f"  - {title} ({score})")
    print()

# 6. Top 5 Movies Overall (by avg rating from real data)
top_5_overall = (
    ratings_df
    .groupby('movie_id')
    .agg(avg_rating=('rating', 'mean'), count=('rating', 'count'))
    .reset_index()
    .sort_values(by='avg_rating', ascending=False)
    .head(5)
)

print("🔥 Top 5 Movies Overall by Average Rating:\n")
for _, row in top_5_overall.iterrows():
    title = movies_df[movies_df.movie_id == row.movie_id]['title'].values[0]
    print(f"{title} — Avg Rating: {round(row.avg_rating, 2)} (based on {row['count']} ratings)")



🎯 Personalized Top 5 Recommendations for Each User:

User 1:
  - The Godfather (4.13)
  - Forrest Gump (4.09)
  - The Notebook (3.97)
  - The Dark Knight (3.97)
  - Superbad (3.97)

User 2:
  - Avengers: Endgame (4.08)
  - Forrest Gump (4.05)
  - Inception (4.01)
  - The Dark Knight (3.96)
  - Superbad (3.89)

User 3:
  - The Godfather (3.97)
  - Superbad (3.94)
  - The Notebook (3.89)
  - The Dark Knight (3.89)
  - Inception (3.85)

User 4:
  - Interstellar (3.93)
  - Inception (3.76)
  - The Notebook (3.72)
  - Avengers: Endgame (3.71)
  - The Godfather (3.63)

User 5:
  - The Notebook (4.01)
  - The Dark Knight (4.01)
  - Avengers: Endgame (3.99)
  - Titanic (3.96)
  - Superbad (3.79)

🔥 Top 5 Movies Overall by Average Rating:

The Notebook — Avg Rating: 5.0 (based on 1.0 ratings)
Avengers: Endgame — Avg Rating: 4.5 (based on 2.0 ratings)
Inception — Avg Rating: 4.5 (based on 2.0 ratings)
The Godfather — Avg Rating: 4.5 (based on 2.0 ratings)
The Dark Knight — Avg Rating: 4.5 (based